<a href="https://colab.research.google.com/github/jgalazka/genelab-tools/blob/main/build_biotype_dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import json

In [2]:
!wget http://ftp.ensembl.org/pub/release-96/gtf/mus_musculus/Mus_musculus.GRCm38.96.gtf.gz

--2021-08-02 20:57:19--  http://ftp.ensembl.org/pub/release-96/gtf/mus_musculus/Mus_musculus.GRCm38.96.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.197.76
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.197.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30085756 (29M) [application/octet-stream]
Saving to: ‘Mus_musculus.GRCm38.96.gtf.gz’

Mus_musculus.GRCm38 100%[===================>]  28.69M   826KB/s    in 37s     

2021-08-02 20:57:57 (788 KB/s) - ‘Mus_musculus.GRCm38.96.gtf.gz’ saved [30085756/30085756]



In [2]:
gtf = pd.read_csv('Mus_musculus.GRCm38.96.gtf.gz', sep='\t', skiprows=5, header=None, low_memory=False)
gtf.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
gtf.head()

,seqname,source,feature,start,end,score,strand,frame,attribute
0,1,havana,gene,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""..."
1,1,havana,transcript,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""..."
2,1,havana,exon,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""..."
3,1,ensembl,gene,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842""; gene_version ""1""..."
4,1,ensembl,transcript,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842""; gene_version ""1""..."


In [3]:
gene_biotype = gtf['attribute'].str.extract(r'gene_biotype \"(\w+)\"')
gtf['gene_biotype'] = gene_biotype

In [4]:
gene_id = gtf['attribute'].str.extract(r'gene_id \"(ENSMUSG\d+)\"')
gtf['gene_id'] = gene_id

In [5]:
gtf.head()

,seqname,source,feature,start,end,score,strand,frame,attribute,gene_biotype,gene_id
0,1,havana,gene,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""...",TEC,ENSMUSG00000102693
1,1,havana,transcript,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""...",TEC,ENSMUSG00000102693
2,1,havana,exon,3073253,3074322,.,+,.,"gene_id ""ENSMUSG00000102693""; gene_version ""1""...",TEC,ENSMUSG00000102693
3,1,ensembl,gene,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842""; gene_version ""1""...",snRNA,ENSMUSG00000064842
4,1,ensembl,transcript,3102016,3102125,.,+,.,"gene_id ""ENSMUSG00000064842""; gene_version ""1""...",snRNA,ENSMUSG00000064842


In [6]:
id_biotype = gtf[['gene_biotype','gene_id']]
id_biotype.head()

,gene_biotype,gene_id
0,TEC,ENSMUSG00000102693
1,TEC,ENSMUSG00000102693
2,TEC,ENSMUSG00000102693
3,snRNA,ENSMUSG00000064842
4,snRNA,ENSMUSG00000064842


In [7]:
# build dictionary linking gene id to biotype
id_biotype_dict = dict(zip(id_biotype.gene_id,id_biotype.gene_biotype))

In [8]:
json = json.dumps(id_biotype_dict)

In [9]:
# export dictionary
f = open("id2biotype.json","w")
f.write(json)
f.close()

In [10]:
# test dictionary
id_biotype_dict['ENSMUSG00000102693']

'TEC'

In [11]:
# check number or rRNA genes
len([k for k,v in id_biotype_dict.items() if v == 'rRNA'])

354